In [4]:

from torch.utils.data import DataLoader
from Controller.TrainingParameters import TrainingParameters
from Utils.Constants import Constants
from enum import Enum

class NotSupportedError(Exception):
    pass

class Datasets(Enum):
    BPI2012 = 1
    Helpdesk = 2

class LossFunc(Enum):
    CrossEntropy = 1

class SelectableModels(Enum):
    BaseLineLSTMModel = 1

class TrainingController:
    def __init__(self, dataset: Datasets, model: SelectableModels):
    
        ## Load dataset
        if (dataset == Datasets.BPI2012):
            self.dataset = BPI2012Dataset(TrainingParameters.bpi_2012_path)
        else:
            raise NotSupportedError("dataset you selected is not supported")

        self.data_loader = DataLoader(self.dataset, batch_size=32, shuffle=True, collate_fn= self.dataset.collate_fn)

        if (model == SelectableModels.BaseLineLSTMModel):
            self.model = BaselineLSTMModel(
                vocab_size= self.dataset.vocab_size(),
                embedding_dim= TrainingParameters.embedding_dim,
                lstm_hidden= TrainingParameters.lstm_hidden,
                dropout= TrainingParameters.dropout,
                num_lstm_layers= TrainingParameters.num_lstm_layers,
                paddingValue= self.dataset.vocab_to_index(Constanst.PAD_VOCAB),
            )
        else:
            raise NotSupportedError("model you selected is not supported")




    